# Explainable AI (XAI) for Electricity Consumption Forecasting Models

In this notebook, we perform residential building electricity consumption forecasting using various ensemble learning models based on decision trees.<br>Since these models are not sensitive to feature scaling, we omit any feature scaling steps.<br>We use two datasets:
- **Household dataset**: Appliances Energy Prediction dataset;
- **University Residential Complex dataset**: Energy consumption data from a university residential complex.

We will:
- Load and preprocess the datasets.
- Train different ensemble models on each dataset.
- Visualize feature importances.
- Use SHAP to explain model predictions.
- Generate high-quality plots (dpi=1000) for detailed analysis.

The figures generated correspond to Figures 4 to 11 in the referenced paper, allowing readers to reproduce and understand the results.

---

**Table of Contents**

1. [Load and Split the Datasets](#load-and-split-the-datasets)
2. [Define Ensemble Models](#define-ensemble-models)
3. [Train Models and Generate Feature Importances](#train-models-and-generate-feature-importances)
4. [SHAP Analysis](#shap-analysis)
5. [Partial Dependence Plots (PDP)](#partial-dependence-plots-pdp)
6. [SHAP Decision Plot](#shap-decision-plot)
7. [Additional Options and Customizations](#additional-options-and-customizations)
8. [Conclusion](#conclusion)
9. [Abbreviation Definitions (Table 2)](#abbreviation-definitions-table-2)

---

**Note:** All plots are saved with a high resolution (dpi=1000) and filenames related to XAI.

In [ ]:
# Install necessary libraries
!pip install shap
!pip install lightgbm
!pip install catboost
!pip install xgboost

# Import libraries
import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

<a name="load-and-split-the-datasets"></a>

## 1. Load and Split the Datasets

We use the `load_and_split_data` function to load either the 'household' or 'dormitory' dataset.<br>The data are split into training and testing sets based on predefined indices.
- household: **Appliances Energy Prediction dataset** 
- dormitory: **University Residential Complex dataset**

### Function Definition

In [ ]:
def load_and_split_data(dataset_choice):
    if dataset_choice == 'household':
        data = pd.read_csv('Appliances Energy Prediction.csv')
        X_train = data.iloc[:2311, 1:-1]
        y_train = data.iloc[:2311, -1]
        X_test = data.iloc[2311:, 1:-1]
        y_test = data.iloc[2311:, -1]
    elif dataset_choice == 'dormitory':
        data = pd.read_csv('University Residential Complex.csv')
        X_train = data.iloc[:20472, 5:-1]
        y_train = data.iloc[:20472, -1]
        X_test = data.iloc[20472:, 5:-1]
        y_test = data.iloc[20472:, -1]
    else:
        raise ValueError("Invalid dataset choice. Please select 'household' or 'dormitory'.")
    return X_train, X_test, y_train, y_test

### Load the Datasets

You can change `dataset_choice` to `'dormitory'` to switch datasets.

In [ ]:
# Select the dataset
dataset_choice = 'dormitory'  # Change to 'household' as needed
X_train_raw, X_test_raw, y_train_raw, y_test_raw = load_and_split_data(dataset_choice)

# Display shapes
print("Training set shape:", X_train_raw.shape, y_train_raw.shape)
print("Test set shape:", X_test_raw.shape, y_test_raw.shape)

<a name="define-ensemble-models"></a>

## 2. Define Ensemble Models

We define a dictionary of ensemble models with their respective hyperparameters for both datasets.<br>The models include:
- **Random Forest Regressor (RF)**;
- **Gradient Boosting Regressor (GBM)**;
- **Extreme Gradient Boosting Regressor (XGBoost)**;
- **Light Gradient Boosting Machine (LightGBM)**;
- **CatBoost Regressor (CatBoost)**.

These models are based on decision trees and do not require feature scaling.

In [ ]:
# Define models for the household dataset
models_household = {
    'RF': RandomForestRegressor(max_features='sqrt', n_estimators=128, random_state=42, n_jobs=-1),
    'GBM': GradientBoostingRegressor(learning_rate=0.05, loss='huber', max_depth=5,
                                     n_estimators=500, random_state=42),
    'XGBoost': XGBRegressor(booster='gbtree', colsample_bytree=1.0, learning_rate=0.05,
                            max_depth=10, n_estimators=1000, subsample=1.0,
                            random_state=42, n_jobs=-1),
    'LightGBM': LGBMRegressor(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.01,
                              n_estimators=1000, num_leaves=64, subsample=0.5,
                              random_state=42, n_jobs=-1),
    'CatBoost': CatBoostRegressor(depth=10, l2_leaf_reg=1, learning_rate=0.03,
                                  loss_function='RMSE', random_seed=42, verbose=False)
}

# Define models for the dormitory dataset
models_dormitory = {
    'RF': RandomForestRegressor(max_features='sqrt', n_estimators=128, random_state=42, n_jobs=-1),
    'GBM': GradientBoostingRegressor(learning_rate=0.05, loss='huber', max_depth=5,
                                     n_estimators=500, random_state=42),
    'XGBoost': XGBRegressor(booster='gbtree', colsample_bytree=1.0, learning_rate=0.05,
                            max_depth=10, n_estimators=1000, subsample=1.0,
                            random_state=42, n_jobs=-1),
    'LightGBM': LGBMRegressor(boosting_type='gbdt', colsample_bytree=1.0, learning_rate=0.01,
                              n_estimators=1000, num_leaves=64, subsample=0.5,
                              random_state=42, n_jobs=-1),
    'CatBoost': CatBoostRegressor(depth=10, l2_leaf_reg=1, learning_rate=0.1,
                                  loss_function='RMSE', random_seed=42, verbose=False)
}

<a name="train-models-and-generate-feature-importances"></a>

## 3. Train Models and Generate Feature Importances

We train each model on the selected dataset and compute the feature importances.<br>We then plot the feature importances using bar plots.

### Training and Feature Importance Plotting

You can adjust the `dataset_choice` variable earlier to switch between datasets.

**Note:** The plots are saved with a high resolution (dpi=1000) for detailed analysis.

---

**Fig 4. Influence of input variables in the GBM model on the University Residential Complex dataset:**

- **(a) Feature importance**

In [ ]:
# Choose the appropriate model dictionary based on the dataset
if dataset_choice == 'household':
    models = models_household
    X_train_data = X_train
    y_train_data = y_train
    X_test_data = X_test
    y_test_data = y_test
    dataset_label = 'Household'
else:
    models = models_dormitory
    X_train_data = X_train
    y_train_data = y_train
    X_test_data = X_test
    y_test_data = y_test
    dataset_label = 'University Residential Complex'

# Dictionary to store trained models
trained_models = {}
feature_importances = {}  # To store feature importances for each model

# Iterate over each model
for model_name, model in models.items():
    print(f"\nTraining {model_name} on {dataset_label} data...")
    model.fit(X_train_data, y_train_data)
    trained_models[model_name] = model

    # Compute feature importances
    if model_name in ['RF', 'GBM', 'XGBoost', 'LightGBM']:
        importances = model.feature_importances_
    elif model_name == 'CatBoost':
        importances = model.get_feature_importance()
    else:
        continue  # Skip if model does not support feature importances

    # Store feature importances
    feature_importances[model_name] = importances

    # Create a DataFrame for feature importances
    fi_df = pd.DataFrame({
        'feature_names': X_train_data.columns,
        'feature_importance': importances
    })

    # Sort the DataFrame by feature importance
    fi_df = fi_df.sort_values(by='feature_importance', ascending=False)

    # Plot feature importances
    plt.figure(figsize=(7, 7))
    sns.barplot(x='feature_importance', y='feature_names', data=fi_df,
                palette=sns.color_palette("husl", len(fi_df)))
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig(f'Fig4a_XAI_Feature_Importance_{model_name}_{dataset_label}.pdf', dpi=1000)
    plt.show()

**Explanation:**

- The above code trains each model and plots the feature importance.
- For **Fig 4(a)**, we focus on the GBM model on the University Residential Complex dataset.
- Ensure that `dataset_choice` is set to `'dormitory'` to reproduce **Fig 4(a)**.

---

**Fig 8. Influence of input variables in the CatBoost model on the Appliances Energy Prediction dataset:**

- **(a) Feature importance**

To reproduce **Fig 8(a)**, set `dataset_choice` to `'household'` and focus on the CatBoost model.

<a name="shap-analysis"></a>

## 4. SHAP Analysis

We use SHAP to explain the predictions of each model.<br>SHAP values represent the contribution of each feature to the prediction made by the model.<br>We generate SHAP summary plots for each model.

### SHAP Summary Plots

**Note:** SHAP plots are saved with a high resolution (dpi=1000) and filenames related to XAI.

---

**Fig 4.**

- **(b) SHAP summary plot for the GBM model on the University Residential Complex dataset.**

In [ ]:
# SHAP analysis for each model
for model_name, model in trained_models.items():
    print(f"\nPerforming SHAP analysis for {model_name} on {dataset_label} data...")
    # Use TreeExplainer for tree-based models
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_train_data)

    # Generate SHAP summary plot
    plt.figure(figsize=(7, 7))
    shap.summary_plot(shap_values, X_train_data, plot_type="dot", show=False)
    plt.tight_layout()
    plt.savefig(f'Fig4b_XAI_SHAP_Summary_{model_name}_{dataset_label}.pdf', dpi=1000)
    plt.show()

**Explanation:**

- For **Fig 4(b)**, we use the GBM model on the University Residential Complex dataset.
- Ensure that `dataset_choice` is `'dormitory'` and `model_name` is `'GBM'`.

---

**Fig 5. SHAP summary plots of other ensemble learning models on the University Residential Complex dataset:**

- **(a) RF**
- **(b) XGBoost**
- **(c) LightGBM**
- **(d) CatBoost**

Repeat the SHAP analysis for the other models (`'RF'`, `'XGBoost'`, `'LightGBM'`, `'CatBoost'`) to generate **Fig 5**, updating filenames accordingly.

---

**Fig 8.**

- **(b) SHAP summary plot for the CatBoost model on the Appliances Energy Prediction dataset.**

Set `dataset_choice` to `'household'` and focus on the `'CatBoost'` model to reproduce **Fig 8(b)**.

---

**Fig 9. SHAP summary plots of other ensemble learning models on the Appliances Energy Prediction dataset:**

- **(a) RF**
- **(b) GBM**
- **(c) XGBoost**
- **(d) LightGBM**

Repeat the SHAP analysis for these models on the `'household'` dataset to generate **Fig 9**, ensuring filenames are appropriately named with 'XAI'.

<a name="partial-dependence-plots-pdp"></a>

## 5. Partial Dependence Plots (PDP)

Partial dependence plots show the relationship between the target response and a set of features of interest.<br> They do this by marginalizing over the values of all other features.

### Generating PDPs Using scikit-learn and SHAP

We compare PDPs generated using scikit-learn's `PartialDependenceDisplay` and SHAP's dependence plots.

---

**Fig 6. Comparison of PDPs from the GBM model on the University Residential Complex dataset:**

- **(a) Analyzing the relationship between holiday status (`Holi`) and average electricity consumption (`Cons_avg`) using scikit-learn**
- **(b) Using SHAP analysis**

In [ ]:
# Select features for PDPs
feature = 'Holi'  # For Fig 6
interaction_feature = 'Cons_avg'  # The feature to analyze interaction with

# Choose the model (GBM in this case)
model_for_pdp = trained_models['GBM']
X_data_for_pdp = X_train_data  # Use training data for PDPs

# Generate PDP using scikit-learn
fig, ax = plt.subplots(figsize=(7, 7))
PartialDependenceDisplay.from_estimator(
    model_for_pdp, X_data_for_pdp, features=[feature], kind="average", ax=ax,
    line_kw={"color": "red"}
)
plt.title(f'PDP using scikit-learn for {feature}')
plt.xlabel(feature)
plt.ylabel('Partial Dependence')
plt.tight_layout()
plt.savefig('Fig6a_XAI_PDP_scikit_Holi_Consavg.pdf', dpi=1000)
plt.show()

# Generate PDP using SHAP
plt.figure(figsize=(7, 7))
shap.dependence_plot(
    ind=feature,
    shap_values=shap_values,
    features=X_data_for_pdp,
    interaction_index=interaction_feature,
    show=False
)
plt.title(f'SHAP Dependence Plot for {feature} and {interaction_feature}')
plt.tight_layout()
plt.savefig('Fig6b_XAI_PDP_SHAP_Holi_Consavg.pdf', dpi=1000)
plt.show()

**Explanation:**

- **Fig 6(a)**: PDP using scikit-learn for 'Holi' on the University Residential Complex dataset.
- **Fig 6(b)**: SHAP dependence plot for 'Holi' interacting with 'Cons_avg'.

---

**Fig 7. Comparison of PDPs from the GBM model on the University Residential Complex dataset:**

- **(a) Analyzing the relationship between temperature-humidity index (`THI`) and `Cons_avg` using scikit-learn**
- **(b) Using SHAP analysis**

Update the `feature` and `interaction_feature`:

In [ ]:
# Update features for Fig 7
feature = 'THI'
interaction_feature = 'Cons_avg'

# Generate PDP using scikit-learn
fig, ax = plt.subplots(figsize=(7, 7))
PartialDependenceDisplay.from_estimator(
    model_for_pdp, X_data_for_pdp, features=[feature], kind="average", ax=ax,
    line_kw={"color": "red"}
)
plt.title(f'PDP using scikit-learn for {feature}')
plt.xlabel(feature)
plt.ylabel('Partial Dependence')
plt.tight_layout()
plt.savefig('Fig7a_XAI_PDP_scikit_THI_Consavg.pdf', dpi=1000)
plt.show()

# Generate PDP using SHAP
plt.figure(figsize=(7, 7))
shap.dependence_plot(
    ind=feature,
    shap_values=shap_values,
    features=X_data_for_pdp,
    interaction_index=interaction_feature,
    show=False
)
plt.title(f'SHAP Dependence Plot for {feature} and {interaction_feature}')
plt.tight_layout()
plt.savefig('Fig7b_XAI_PDP_SHAP_THI_Consavg.pdf', dpi=1000)
plt.show()

**Explanation:**

- **Fig 7(a)**: PDP using scikit-learn for 'THI' on the University Residential Complex dataset.
- **Fig 7(b)**: SHAP dependence plot for 'THI' interacting with 'Consavg'.

---

**Note:** Ensure that `dataset_choice` is `'dormitory'` and `model_name` is `'GBM'` to reproduce Figures 6 and 7.<br>The filenames include 'XAI' to indicate their relation to explainable AI methods.

<a name="shap-decision-plot"></a>

## 6. SHAP Decision Plot

SHAP decision plots visualize model predictions by highlighting how the SHAP values of each feature contribute to the final prediction.

### Generating SHAP Decision Plot on Test Set

We apply the SHAP decision plot to the test set to see how the model makes predictions on unseen data.

---

**Fig 11. SHAP decision plot for the test set on the Appliances Energy Prediction dataset.**

In [ ]:
# Switch to Household dataset to reproduce Fig 11
dataset_choice = 'household'
X_train, X_test, y_train, y_test = load_and_split_data(dataset_choice)

# Train the CatBoost model
model_catboost_household = models_household['CatBoost']
model_catboost_household.fit(X_train, y_train)

# Compute SHAP values on test set
explainer_test = shap.TreeExplainer(model_catboost_household)
shap_values_test = explainer_test.shap_values(X_test)

# Generate SHAP decision plot
plt.figure(figsize=(15, 7.5))
expected_value = explainer_test.expected_value
if isinstance(expected_value, np.ndarray):
    expected_value = expected_value[0]  # For some models, expected_value is an array
shap.decision_plot(
    expected_value, shap_values_test[:100], X_test.iloc[:100], feature_order='hclust',
    show=False
)
plt.title('Fig11_XAI_SHAP Decision Plot on Test Set (Appliances Energy Prediction dataset)')
plt.tight_layout()
plt.savefig('Fig11_XAI_SHAP_Decision_Plot_Household.pdf', dpi=1000)
plt.show()

**Explanation:**

- We apply the SHAP decision plot to the test set of the Appliances Energy Prediction dataset.
- The plot shows how each feature contributes to the model's predictions on unseen data.
- We limit the plot to the first 100 instances for clarity.

---

**Note:** Adjust `dataset_choice` and the model accordingly to reproduce **Fig 11**.

<a name="additional-options-and-customizations"></a>

## 7. Additional Options and Customizations

To help readers adjust the code and generate specific figures:
- **Adjust Dataset Choice**: Change `dataset_choice` between `'household'` and `'dormitory'` to switch datasets.
- **Select Specific Models**: Modify the `models` dictionary or specify `model_name` to focus on specific models.
- **Change Features for PDPs**: Update `feature` and `interaction_feature` with desired feature names.
- **Customize Plots**: Modify plot sizes (`figsize`), titles, labels, and other parameters in the plotting functions.
- **SHAP Options**: Explore different SHAP plots such as `bar`, `violin`, and `beeswarm` plots by changing the `plot_type` parameter.
- **Filename Adjustments**: Ensure filenames include 'XAI' to reflect their focus on explainable AI.

### Generating Figures 5, 9, and 10

**Fig 5** and **Fig 9** correspond to SHAP summary plots of other ensemble learning models.<br>You can generate these by iterating over the models as shown in the SHAP Analysis section, ensuring filenames include 'XAI'.

**Fig 10** involves PDPs of the CatBoost model on the Appliances Energy Prediction dataset:

---

**Fig 10. PDPs of the CatBoost model on the Appliances Energy Prediction dataset:**
- **(a) Analyzing the interaction between `Hour_x` and `Hour_y`**
- **(b) Analyzing the interaction between `Holi` and `Hour_y`**

In [ ]:
```python
# Ensure dataset_choice is 'household'
dataset_choice = 'household'
X_train, X_test, y_train, y_test = load_and_split_data(dataset_choice)

# Use the trained CatBoost model
model_catboost_household = models_household['CatBoost']
model_catboost_household.fit(X_train, y_train)

# Compute SHAP values on training set
explainer_catboost = shap.TreeExplainer(model_catboost_household)
shap_values_catboost = explainer_catboost.shap_values(X_train)

# Generate PDPs for 'Hour_x' and 'Hour_y', 'Holi' and 'Hour_y'
features_pairs = [('Hour_x', 'Hour_y'), ('Holi', 'Hour_y')]

for features in features_pairs:
    # SHAP dependence plot with interaction
    plt.figure(figsize=(7, 7))
    shap.dependence_plot(
        ind=features[0],
        shap_values=shap_values_catboost,
        features=X_train,
        interaction_index=features[1],
        show=False
    )
    plt.title(f'SHAP Dependence Plot for {features[0]} and {features[1]}')
    plt.tight_layout()
    plt.savefig(f'Fig10_XAI_SHAP_Dependence_{features[0]}_{features[1]}_Household.pdf', dpi=1000)
    plt.show()

**Explanation:**

- This code generates SHAP dependence plots analyzing the interaction between specified features.
- The plots correspond to **Fig 10(a)** and **Fig 10(b)**.
- Filenames include 'XAI' to reflect their focus on explainable AI methods.

---
<a name="conclusion"></a>

## Conclusion

In this notebook, we:
- Trained various ensemble models on the selected datasets without feature scaling, as they are based on decision trees.
- Analyzed feature importances to understand which features contribute most to the models.
- Used SHAP to explain individual predictions and overall feature impact.
- Generated high-quality visualizations for detailed analysis, corresponding to Figures 4 to 11.
- Provided options and detailed explanations to help readers reproduce and customize the analysis.

This comprehensive approach helps in understanding both the performance of the models and the factors influencing their predictions.<br>This insight is crucial for making informed decisions in energy management.

## Table 2: Abbreviation Definitions

To understand the abbreviations used in the figures:

<a name="abbreviation-definitions-table-2"></a>

| Variable | Description                                        | Data Type                           |
|:----------|:----------------------------------------------------|:-------------------------------------|
| **Timestamp Features**                                        |                                     |
| Hour_x    | Hour of the day (sine value)                       | Numeric (Timestamp)                 |
| Hour_y    | Hour of the day (cosine value)                     | Numeric (Timestamp)                 |
| DOTW_x    | Day of the week (sine value)                       | Numeric (Timestamp)                 |
| DOTW_y    | Day of the week (cosine value)                     | Numeric (Timestamp)                 |
| Holi     | Weekday: 0; Weekend: 1                             | Binary (Timestamp)                  |
| **Weather Conditions**                                        |                                     |
| Temp     | Hourly temperature                                 | Numeric (Weather condition)         |
| Humi     | Hourly humidity                                    | Numeric (Weather condition)         |
| WS       | Hourly wind speed                                  | Numeric (Weather condition)         |
| THI      | Hourly temperature-humidity index                  | Numeric (Weather condition)         |
| WCT      | Hourly wind chill temperature                      | Numeric (Weather condition)         |
| **Historical Electricity Consumption**                        |                                     |
| Cons_1    | Power consumption one day before the time point    | Numeric (Historical consumption)    |
| Holi_1    | Holiday indicator one day before the time point    | Binary (Historical consumption)     |
| Cons_7    | Power consumption one week before the time point   | Numeric (Historical consumption)    |
| Holi_7    | Holiday indicator one week before the time point   | Binary (Historical consumption)     |
| Cons_avg  | Weekly average power consumption                   | Numeric (Historical consumption)    |

**Note:** This table provides definitions for the variables used in the datasets and figures.

---

**Note to Readers:** Feel free to modify the code to explore different models, datasets, and features.<br>The detailed explanations and code comments are provided to facilitate understanding and customization.

---

**End of Notebook**